In [1]:
%run ../algorithmeX.ipynb

## [Polycubes](https://fr.wikipedia.org/wiki/Polycube)
Pour $i,j,k\in \mathbf Z$, appelons *cube* $(i,j,k)$ le cube de l'espace dont les sommets sont les $(i+\delta i,j+\delta j,k+\delta k)$ où $\delta i,\delta j,\delta k\in\{0,1\}$.


On représente un polycube par une instance de la classe $\texttt{PIECE}$ :

*nom* $\texttt{ = PIECE('}\!$ *nom* $\!\texttt{',}$ *piece* $\texttt{,}$ *couleur*  $\texttt{)}$

où *piece* est un ensemble (type $\texttt{frozenset}$ et non  $\texttt{set}$) de couples d'entiers $(i,j,k)$ qui représente la forme du polycube.  
Il est important que le nom donné à l'instance soit le même que celui passé en argument de  $\texttt{\_\_init\_\_}$

Pour être placé dans un volume de puzzle 3D, un polycube doit pouvoir être tourné. 

Le groupe $G$ des isométries directes du cube est formé des applications $(x_0,x_1,x_2)\mapsto(\varepsilon_0x_{\sigma(0)}, \varepsilon_1x_{\sigma(1)}, \varepsilon_2x_{\sigma(2)})$ où $\sigma\in\mathfrak S_3$ est une permutation de $\{0,1,2\}$ et 
$(\varepsilon_0, \varepsilon_1, \varepsilon_2)\in\{\pm1\}^3$ satisfont 
signature $\!(\sigma)\times\varepsilon_0\times\varepsilon_1\times\varepsilon_2=1$.

En effet, on peut vérifier que ces $24$ applications sont bien dans $G$ et on sait que $G$ comporte $24$ éléments car il est isomorphe à $\mathfrak S_4$.

*nom* $\texttt{\!.isometriques()}$ renvoie la liste des $\leqslant 24$ polycubes obtenus en faisant tourner *nom*.
   


In [2]:
import random

snsSet3Set2Colors = [(0.5529411764705883, 0.8274509803921568, 0.7803921568627451), (1.0, 1.0, 0.7019607843137254), (0.7450980392156863, 0.7294117647058823, 0.8549019607843137), (0.984313725490196, 0.5019607843137255, 0.4470588235294118), (0.5019607843137255, 0.6941176470588235, 0.8274509803921568), (0.9921568627450981, 0.7058823529411765, 0.3843137254901961), (0.7019607843137254, 0.8705882352941177, 0.4117647058823529), (0.9882352941176471, 0.803921568627451, 0.8980392156862745), (0.8509803921568627, 0.8509803921568627, 0.8509803921568627), (0.7372549019607844, 0.5019607843137255, 0.7411764705882353), (0.8, 0.9215686274509803, 0.7725490196078432), (1.0, 0.9294117647058824, 0.43529411764705883), (0.4, 0.7607843137254902, 0.6470588235294118), (0.9882352941176471, 0.5529411764705883, 0.3843137254901961), (0.5529411764705883, 0.6274509803921569, 0.796078431372549), (0.9058823529411765, 0.5411764705882353, 0.7647058823529411), (0.6509803921568628, 0.8470588235294118, 0.32941176470588235), (1.0, 0.8509803921568627, 0.1843137254901961), (0.8980392156862745, 0.7686274509803922, 0.5803921568627451), (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]
snsPastelColors = [(0.984313725490196, 0.7058823529411765, 0.6823529411764706), (0.7019607843137254, 0.803921568627451, 0.8901960784313725), (0.8, 0.9215686274509803, 0.7725490196078432), (0.8705882352941177, 0.796078431372549, 0.8941176470588236), (0.996078431372549, 0.8509803921568627, 0.6509803921568628), (1.0, 1.0, 0.8), (0.8980392156862745, 0.8470588235294118, 0.7411764705882353), (0.9921568627450981, 0.8549019607843137, 0.9254901960784314), (0.9490196078431372, 0.9490196078431372, 0.9490196078431372), (0.7019607843137254, 0.8862745098039215, 0.803921568627451), (0.9921568627450981, 0.803921568627451, 0.6745098039215687), (0.796078431372549, 0.8352941176470589, 0.9098039215686274), (0.9568627450980393, 0.792156862745098, 0.8941176470588236), (0.9019607843137255, 0.9607843137254902, 0.788235294117647), (1.0, 0.9490196078431372, 0.6823529411764706), (0.9450980392156862, 0.8862745098039215, 0.8), (0.8, 0.8, 0.8)]
snsTab20Colors = [(0.12156862745098039, 0.4666666666666667, 0.7058823529411765), (0.6823529411764706, 0.7803921568627451, 0.9098039215686274), (1.0, 0.4980392156862745, 0.054901960784313725), (1.0, 0.7333333333333333, 0.47058823529411764), (0.17254901960784313, 0.6274509803921569, 0.17254901960784313), (0.596078431372549, 0.8745098039215686, 0.5411764705882353), (0.8392156862745098, 0.15294117647058825, 0.1568627450980392), (1.0, 0.596078431372549, 0.5882352941176471), (0.5803921568627451, 0.403921568627451, 0.7411764705882353), (0.7725490196078432, 0.6901960784313725, 0.8352941176470589), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354), (0.7686274509803922, 0.611764705882353, 0.5803921568627451), (0.8901960784313725, 0.4666666666666667, 0.7607843137254902), (0.9686274509803922, 0.7137254901960784, 0.8235294117647058), (0.4980392156862745, 0.4980392156862745, 0.4980392156862745), (0.7803921568627451, 0.7803921568627451, 0.7803921568627451), (0.7372549019607844, 0.7411764705882353, 0.13333333333333333), (0.8588235294117647, 0.8588235294117647, 0.5529411764705883), (0.09019607843137255, 0.7450980392156863, 0.8117647058823529), (0.6196078431372549, 0.8549019607843137, 0.8980392156862745)]

palette = snsPastelColors


def randomColor(seed = None):
    if seed is not None:
        random.seed(str(seed))
    return random.choice(palette)

In [3]:
class PIECE:

    def __init__(self,nom,piece,couleur = None):
        self.nom = nom
        self.piece = PIECE.normalise(piece)
        self.couleur = couleur
        self.largeur = max(t[0] for t in self.piece) + 1
        self.profondeur = max(t[1] for t in self.piece) + 1
        self.hauteur = max(t[2] for t in self.piece) + 1

    def __hash__(self):
        return hash(self.piece)

    def __eq__(self,q):
        return self.piece == q.piece

    def __str__(self):
        return str((self.nom,self.piece))
 
    def normalise(piece):
        min_x = min(t[0] for t in piece)
        min_y = min(t[1] for t in piece)
        min_z = min(t[2] for t in piece)
        return frozenset((x - min_x, y - min_y, z - min_z) for x, y, z in piece)

    def isometriques(self):
        l = []
        for p in Permutations(3):
            q = Permutation(p)
            u0, u1, u2 = q
            u = u0 - 1, u1 - 1, u2 - 1
            for e0 in [-1,1]:
                for e1 in [-1,1]:
                    for e2 in [-1,1]:
                        if q.signature() * e0 * e1 * e2 == 1:
                            def sigma(t):
                                return (e0 * t[u[0]], e1 * t[u[1]], e2 * t[u[2]])
                            l.append(PIECE(self.nom, frozenset(sigma(t) for t in self.piece))) 
        return list(set(l))
    
    def plot(self):
        G = Graphics()    
        couleur = 'white' if self.couleur is None else self.couleur    
        for i,j,k in self.piece:
            G += Polyhedron(vertices = [(i+di,j+dj,k+dk) 
                    for di in [0,1] for dj in [0,1] for dk in [0,1] ]).plot(color = couleur)
        G.show(frame = False)        

#### Génération des polycubes de taille donnée
$N=6$

On calcule une liste $\texttt{polycubes}$ telle que, pour $1\leqslant n\leqslant N$, $\texttt{polycubes[}n\texttt{]}=$ la liste des polycubes de taille $n$.

In [4]:
N = 6 # 2 s (N = 7 : 7 s, N = 8 : 46 s, N = 9 : 6 mns) 

def genPolys(p):
    """
    p est l'ensemble des x.piece quand x parcourt l'ensembles des polycubes de taille n
    genPolys(p) renvoie  l'ensemble des x.piece quand x parcourt l'ensembles des polycubes de taille n + 1
    """
    q = set()
    for piece0 in p:
        for i0,j0,k0 in piece0:
            for i, j, k in [(i0, j0 + 1, k0), (i0, j0 - 1, k0), (i0 + 1, j0, k0), (i0 - 1, j0, k0), (i0, j0, k0 + 1), (i0, j0, k0 - 1)]:
                if (i, j, k) not in piece0:
                    Piece = PIECE('', piece0 | frozenset([(i, j, k)]))
                    if q.isdisjoint(Piece.isometriques()):
                        q.add(Piece)
    return {x.piece for x in q}

_p = [set(),{frozenset([(0,0,0)])}]
for n in range(N - 1): 
    _p.append(genPolys(_p[-1]))
    
polycubes = [[PIECE(f'p{i}_{j}', piece, couleur=palette[j%(len(palette))]) 
                for j, piece in enumerate(sorted(list(e),key = hash))]
                for i, e in enumerate(_p)]

for e in polycubes: 
    for p in e: globals()[p.nom] = p    
    
prefixeNumerique = {1: 'mono', 2 : 'di', 3: 'tri', 4 : 'tétra', 5 : 'penta', 
                      6 : 'hexa', 7 : 'hepta', 8 : 'octa', 9 : 'nona', 10 : 'déca'}

for i in range(1,N + 1):
    print(f'{i} : {len(polycubes[i]):>3} {prefixeNumerique[i]}cubes')


1 :   1 monocubes
2 :   1 dicubes
3 :   2 tricubes
4 :   8 tétracubes
5 :  29 pentacubes
6 : 166 hexacubes


Les 7 plus petits polycubes non cuboidaux (somas)

In [5]:
#------------------------------- Les 7 plus petits polycubes non cuboidaux (somas) ----------------------------

Bent = PIECE( 'Bent',((0,0,0), (0,1,0), (1,0,0)), couleur = palette[0])
Ell = PIECE('Ell',((0,0,0), (0,1,0), (1,0,0), (2,0,0)), couleur = palette[1])
Tee = PIECE('Tee',((-1,0,0), (0,0,0), (1,0,0), (0,1,0)), couleur = palette[2])
Skew = PIECE('Skew',((0,0,0), (1,0,0), (1,1,0), (2,1,0)), couleur = palette[3])
Ltwist = PIECE('Ltwist',((0,0,0), (0,1,0), (1,0,0),(1,0,1)), couleur = palette[4])
Rtwist = PIECE('Rtwist',((0,0,0), (0,1,0), (1,0,0),(0,1,1)), couleur = palette[5])
Claw = PIECE('Claw',((0,0,0), (0,1,0), (1,0,0),(0,0,1)), couleur = palette[6])

In [6]:
[len(p.isometriques()) for p in [Bent, Ell, Tee, Skew, Ltwist, Rtwist, Claw]]

[12, 24, 12, 12, 12, 12, 8]

In [7]:

class PUZZLE:

    def __init__(self,pieces,
                      min_i,max_i,min_j,max_j,min_k,max_k,
                      conditions = None,
                      strict = True):
        """pieces : liste de PIECE
        min_i,max_i,min_j,max_j,min_k,max_k : definition du volume à remplir
        conditions : triplet d'entiers -> booleen restreignant le volume
    
        1er cas  : strict = True
            Pour chacun des noms des pieces, il faut placer 
            une et une seule piece portant ce nom.
            Les elements de E sont les noms des pieces
            et les cases (triplets d'entiers) du volume.
            Chaque element de F contient un et un seul nom
            et les cases du plateau utilisees par une piece de ce nom..
        
        2eme cas : strict = False
            Pour chaque piece de pieces, on dispose,
            pour resoudre le puzzle, d'autant d'exemplaires
            que l'on veut de la piece.
            Les elements de E sont les cases du volume.
        """
        self.min_i = min_i
        self.max_i = max_i
        self.min_j = min_j
        self.max_j = max_j
        self.min_k = min_k
        self.max_k = max_k
        self.strict = strict
        self.pieces = pieces
        if not conditions:
            conditions = lambda i,j,k: True
        lignes = dict()
        nbLignes = 0
        for p in pieces:
            for u in range(min_i, max_i - p.largeur + 2):
                for v in range(min_j, max_j - p.profondeur + 2):
                    for w in range(min_k, max_k - p.hauteur + 2):
                        ligne = [p.nom] if self.strict else []
                        match = True
                        for i,j,k in p.piece:
                            ic, jc, kc  = i + u, j + v, k + w
                            if conditions(ic,jc,kc):
                                ligne.append((ic,jc,kc))
                            else:
                                match = False
                                break
                        if match:
                            lignes[nbLignes] = ligne
                            nbLignes += 1
        self.lignes = lignes
        
    def solve(self):

        F = self.lignes
        return AlgorithmeX(F).solve()

    def printSolution(self,sol):     
        for l in sol: print(self.lignes[l])

    def plotSolution(self,sol,ecart=0,**kwargs):
        
        if not self.strict:
            random.seed('314')
        G = Graphics()
        for l in sol:
            e = set()
            for c in self.lignes[l]:
                if type(c) == tuple:
                    e.add(c)
                else:
                    couleur = eval(c).couleur 
            if not self.strict: couleur = randomColor()  
            couche = min([k for i,j,k in e])          
            for i,j,k in e:
                G += Polyhedron(vertices = [(i+di,j+dj,k+dk+couche*ecart) for di in [0,1] for dj in [0,1] for dk in [0,1] ]).plot(color = couleur, **kwargs)
   
        G.show(frame = False)


In [8]:
#from sage.plot.plot3d.texture import Texture
# Texture(id, color=(0.4, 0.4, 1), opacity=1, ambient=0.5, diffuse=1, specular=0, shininess=1, name=None, **kwds)

def somas(*args):
    puzzle = PUZZLE(Ell.isometriques() + Bent.isometriques() + Tee.isometriques() + Skew.isometriques() + \
                            Ltwist.isometriques() + Rtwist.isometriques() + Claw.isometriques(),*args)
    sols = puzzle.solve()
    s = next(sols)
    puzzle.plotSolution(s)

somas(1,3,1,3,1,3)
somas(1,4,1,3,1,3,lambda i,j,k: (i,j) not in [(1,1),(1,3),(4,2)])
somas(1,5,1,3,1,2,lambda i,j,k:(i,j,k) not in [(1,2,2),(3,2,2),(5,2,2)])

Graphics3d Object

Graphics3d Object

Graphics3d Object

In [9]:
def unePiece(piece,ecart,p,q,r):
    puzzle = PUZZLE(piece.isometriques(),1,p,1,q,1,r,strict = False)    
    sol = puzzle.solve()
    s = next(sol)
    puzzle.plotSolution(s, ecart = ecart)

unePiece(Ltwist,2,4,4,4)

Graphics3d Object

In [10]:
unePiece(Claw,0,4,4,2)

Graphics3d Object

In [11]:
unePiece(Skew,3,4,4,4)

Graphics3d Object

In [12]:
unePiece(Bent,2,3,3,3)

Graphics3d Object

In [13]:
# 2528 solutions essentielles

Arthur = PIECE('Arthur',((-1,0,0), (0,0,0), (0,1,0), (1,0,0), (2,0,0)))

unePiece(Arthur,0,5,5,5)

Graphics3d Object

In [14]:
unePiece(Arthur,4,5,5,5)

Graphics3d Object

Un [cube 6x6x6](https://www.gamepuzzles.com/hexacube.htm) construit à l'aide des 35 hexacubes planaires et d'un hexacube non planaire.

In [15]:
def isPlanar(polycube):
    return polycube.largeur == 1 or polycube.profondeur == 1 or polycube.hauteur == 1

hexaplanars = [polycube for polycube in polycubes[6] if isPlanar(polycube)]
hexaNonPlanar = [polycube for polycube in polycubes[6] if polycube not in hexaplanars][0]

l = [hexaNonPlanar]

for polycube in hexaplanars: l += polycube.isometriques()
puzzle = PUZZLE(l, 1,6,1,6,1,6)
sols = puzzle.solve()
s = next(sols) # 5mns 30s
puzzle.plotSolution(s)

Graphics3d Object

In [ ]:
puzzle.plotSolution(s, ecart = 4)

Un [cube 10x10x10](https://www.gamepuzzles.com/hexacube.htm) construit à l'aide des 166 hexacubes et de 4 monocubes.

Malheureusement, le code suivant fait crasher le noyau ...

In [ ]:
Cube1 = PIECE('Cube1', {(0,0,0)})
Cube2 = PIECE('Cube2', {(0,0,0)})
Cube3 = PIECE('Cube3', {(0,0,0)})
Cube4 = PIECE('Cube4', {(0,0,0)})
l = [Cube1, Cube2, Cube3, Cube4]
for p in polycubes[6]: l += p.isometriques()
puzzle = PUZZLE(l, 1,10,1,10,1,10)
sols = puzzle.solve()
s = next(sols)
puzzle.plotSolution(s)